In [10]:
# Import required libraries
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio.tools as tls

# Set your Plotly Chart Studio credentials
tls.set_credentials_file(username='MyUsername', api_key='MyAPIKey')

# Assign the Kaggle csv file to our Pandas dataframe
df = pd.read_csv("Methane_final.csv", index_col=0)

In [ ]:
# Check for null values
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1548 entries, 0 to 1547
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   region     1548 non-null   object 
 1   country    1548 non-null   object 
 2   emissions  1548 non-null   float64
 3   type       1548 non-null   object 
 4   segment    1548 non-null   object 
 5   reason     1548 non-null   object 
 6   baseYear   1548 non-null   object 
 7   notes      1548 non-null   object 
dtypes: float64(1), object(7)
memory usage: 108.8+ KB


In [11]:
df

,region,country,emissions,type,segment,reason,baseYear,notes
0,Africa,Algeria,257.611206,Agriculture,Total,All,2019-2021,Average based on United Nations Framework Conv...
1,Africa,Algeria,0.052000,Energy,Bioenergy,All,2022,Estimates from end-uses are for 2020 or 2021 (...
2,Africa,Algeria,130.798996,Energy,Gas pipelines and LNG facilities,Fugitive,2022,Not available
3,Africa,Algeria,69.741898,Energy,Gas pipelines and LNG facilities,Vented,2022,Not available
4,Africa,Algeria,213.987000,Energy,Onshore gas,Fugitive,2022,Not available
...,...,...,...,...,...,...,...,...
1543,World,World,3102.500000,Energy,Satellite-detected large oil and gas emissions,All,2022,Not available
1544,World,World,30296.500000,Energy,Steam coal,All,2022,Not available
1545,World,World,133350.984375,Energy,Total,All,2022,Estimates from end-uses are for 2020 or 2021 (...
1546,World,World,9737.874023,Other,Total,All,2019-2021,Average based on United Nations Framework Conv...


In [3]:
# I noticed that the last of the entries were world totals for different industry types, 
# so we need to get rid of any rows containing the country "World"

df = df[df['country'] != 'World']
df = df[df['segment'] == 'Total']

In [4]:
# Reassign only the columns of interest to the dataframe 
columns_of_interest = ['country', 'emissions', 'type']
df = df[columns_of_interest]

# Need to take the sum of emissions for each country to produce a unique value per country
df_total_emissions = df.groupby('country')['emissions'].sum().reset_index()

df_total_emissions

# Find the country with max emissions and normalize the summed emissions data to that country's value
max_emissions = df_total_emissions['emissions'].max()
df_total_emissions['normalized_emissions'] = df_total_emissions['emissions'] / max_emissions

df_total_emissions

,country,emissions,normalized_emissions
0,Algeria,3444.247114,0.061862
1,Angola,1202.906883,0.021605
2,Argentina,5251.163147,0.094316
3,Australia,5543.555191,0.099568
4,Azerbaijan,623.687966,0.011202
...,...,...,...
99,Uruguay,901.324079,0.016189
100,Uzbekistan,1966.862615,0.035327
101,Venezuela,4048.541238,0.072716
102,Vietnam,3072.738373,0.055189


In [14]:
# Create a world map with choropleth coloring
fig = go.Figure(data=go.Choropleth(
    locations=df_total_emissions['country'],
    locationmode='country names',
    z=df_total_emissions['normalized_emissions'],
    text=df_total_emissions['emissions'],
    hovertemplate='<b>%{location}</b><br>' +
                  'Total Methane Emissions (Mt): %{text:,.0f}<extra></extra>',
    colorscale=[[0, 'green'], [1, 'red']],
    colorbar_title='Emissions',
))

# Set plot title
fig.update_layout(
    title_text='Total Methane Emissions by Country (Normalized)',
    geo=dict(showframe=False, showcoastlines=False, projection_type='natural earth'),
    width=800,  # Set the width of the map
    height=600,  # Set the height of the map
)

# Zoom in and center on the globe
fig.update_geos(
    fitbounds='locations',
    visible=False,
    projection_type='orthographic',
    lonaxis_range=[-180, 180],
    lataxis_range=[-90, 90],
    showocean=True,
    oceancolor='rgb(204, 230, 255)',
)

chart_studio_url = py.plot(fig, auto_open=False)

# Display the Chart Studio URL
print(f"Interactive map URL: {chart_studio_url}")

Interactive map URL: https://plotly.com/~lukasj123/19/
